In [36]:

if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
    BiocManager::install(version = "3.13")

BiocManager::install("TxDb.Hsapiens.UCSC.hg19.knownGene")
BiocManager::install("rtracklayer")
BiocManager::install("GenomicRanges")
library(GenomicRanges)
library(rtracklayer)
test_path <- system.file("home/jovyan/work/imr_grohmmfinal.transcripts.bed", package = "rtracklayer")
test_bed <- file.path(test_path, "home/jovyan/work/imr_grohmmfinal.transcripts.bed")
test <- import(test_bed)
test
test_bed_file <- BEDFile(test_bed)
import(test_bed_file)
test_bed_con <- file(test_bed)

#groHMM data (starting with bed file of txs)

#create GRanges of annotated regions in hg19
library(GenomicRanges)
library(rtracklayer)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
txdb <- TxDb.Hsapiens.UCSC.hg19.knownGene
tx <- transcripts(txdb)  

#read in final tx bed files from groHMM
txIMR <- rtracklayer::import("imr_grohmmfinal.transcripts.bed")
txIMR$name <- paste0(seqnames(txIMR),"-" , start(txIMR), strand(txIMR))
txGM <- rtracklayer::import("gm_grohmmfinal.transcripts.bed")
txGM$name <- paste0(seqnames(txGM),"-" , start(txGM), strand(txGM))

#subset TX list by intergenic regions
txIMR_int <- subsetByOverlaps(txIMR, tx, type = "any", invert = T)
txGM_int <- subsetByOverlaps(txGM, tx, type = "any", invert = T)

#further subset by overlap with histone marks
#create GRanges for histone bed files
#to make import work, I had to delete the "score" column (5) and chrM entries in the IMR90 bed files
bed <- read.table("GSM469967_UCSD.IMR90.H3K27ac.LL235.bed.gz")
bed$V5 <- NULL
bed <- filter(bed, V1 != "chrM")
write.table(bed, "GSM469967_UCSD.IMR90.H3K27ac.LL235.bed", quote = F, sep="\t", row.names=F, col.names=F)

bed <- read.table("GSM521898_UCSD.IMR90.H3K4me1.SYL133.bed.gz")
bed$V5 <- NULL
bed <- filter(bed, V1 != "chrM")
write.table(bed, "GSM521898_UCSD.IMR90.H3K4me1.SYL133.bed", quote = F, sep="\t", row.names=F, col.names=F)
rm(bed)

IMR_H3K27ac <- rtracklayer::import("GSM469967_UCSD.IMR90.H3K27ac.LL235.bed",format = "bed")
IMR_H3K4me1 <- rtracklayer::import("GSM521898_UCSD.IMR90.H3K4me1.SYL133.bed",format = "bed")

extraCols <- c(signalValue = "numeric", pValue = "numeric", qValue = "numeric")
GM_H3K4me1 <- rtracklayer::import("GSM733772_GM_H3K4me1.broadPeak",format = "bed", extraCols = extraCols)
GM_H3K27ac <- rtracklayer::import("GSM733771_GM_H3K27ac.broadPeak",format = "bed", extraCols = extraCols)

txIMR_eRNA <- subsetByOverlaps(txIMR_int, IMR_H3K27ac)
txIMR_eRNA <- subsetByOverlaps(txIMR_eRNA, IMR_H3K4me1)
export(txIMR_eRNA, "IMR_eRNA.gtf")

txGM_eRNA <- subsetByOverlaps(txGM_int, GM_H3K27ac)
txGM_eRNA <- subsetByOverlaps(txGM_eRNA, GM_H3K4me1)
export(txGM_eRNA, "GM_eRNA.gtf")

#feature counts from groHMM results
library(Rsubread)
bamsIMR <- list.files( "./bam_files", pattern=glob2rx("*IMR*bam$"), full=TRUE )
bamsGM <- list.files( "./bam_files", pattern=glob2rx("*GM*bam$"), full=TRUE )
geneCtsIMR <- featureCounts(files = bamsIMR, annot.inbuilt = "hg19", useMetaFeatures = T, strandSpecific = 1, nthreads = 8)
geneCtsGM <- featureCounts(files = bamsGM, annot.inbuilt = "hg19", useMetaFeatures = T, strandSpecific = 1, nthreads = 8)
eRNACtsGM <- featureCounts(files = bamsGM, annot.ext = "GM_eRNA.gtf", isGTFAnnotationFile = T, GTF.featureType = "sequence_feature", GTF.attrType = "name", strandSpecific = 1, nthreads = 8)
eRNACtsIMR <- featureCounts(files = bamsIMR, annot.ext = "IMR_eRNA.gtf", isGTFAnnotationFile = T, GTF.featureType = "sequence_feature", GTF.attrType = "name", strandSpecific = 1, nthreads = 8)

#create count matrix
#geneCts[4] provides quality/mapping info
IMRc1 <- as.data.frame(geneCtsIMR[1])
IMRc2 <- as.data.frame(geneCtsIMR[2])
mRNAimr <- cbind(IMRc1, IMRc2)

IMRc1 <- as.data.frame(eRNACtsIMR[1])
IMRc2 <- as.data.frame(eRNACtsIMR[2])
eRNAimr <- cbind(IMRc1, IMRc2)
rm(IMRc1, IMRc2)

GMc1 <- as.data.frame(geneCtsGM[1])
GMc2 <- as.data.frame(geneCtsGM[2])
mRNAgm <- cbind(GMc1, GMc2)

GMc1 <- as.data.frame(eRNACtsGM[1])
GMc2 <- as.data.frame(eRNACtsGM[2])
eRNAgm <- cbind(GMc1, GMc2)
rm(GMc1, GMc2)

#clean up count files
library(stringr)
colnames(mRNAimr)[13:18] <- sub("annotation.", "", colnames(mRNAimr)[13:18])
colnames(mRNAimr)[1:12] <- sub(c("counts."), "", colnames(mRNAimr)[1:12])
colnames(mRNAimr)[1:12] <- sub(c(".bam"), "", colnames(mRNAimr)[1:12])

colnames(mRNAgm)[13:18] <- sub("annotation.", "", colnames(mRNAgm)[13:18])
colnames(mRNAgm)[1:12] <- sub(c("counts."), "", colnames(mRNAgm)[1:12])
colnames(mRNAgm)[1:12] <- sub(c(".bam"), "", colnames(mRNAgm)[1:12])

colnames(eRNAimr)[13:18] <- sub("annotation.", "", colnames(eRNAimr)[13:18])
colnames(eRNAimr)[1:12] <- sub(c("counts."), "", colnames(eRNAimr)[1:12])
colnames(eRNAimr)[1:12] <- sub(c(".bam"), "", colnames(eRNAimr)[1:12])

colnames(eRNAgm)[13:18] <- sub("annotation.", "", colnames(eRNAgm)[13:18])
colnames(eRNAgm)[1:12] <- sub(c("counts."), "", colnames(eRNAgm)[1:12])
colnames(eRNAgm)[1:12] <- sub(c(".bam"), "", colnames(eRNAgm)[1:12])

mRNAimr$Chr <- word(mRNAimr$Chr, 1, sep = "\\;")
mRNAimr$Start <- word(mRNAimr$Start, 1, sep = "\\;")
mRNAimr$End <- word(mRNAimr$End, 1, sep = "\\;")
mRNAimr$Strand <- word(mRNAimr$Strand, 1, sep = "\\;")
mRNAimr$Start <- as.numeric(mRNAimr$Start)
mRNAimr$End <- as.numeric(mRNAimr$End)

mRNAgm$Chr <- word(mRNAgm$Chr, 1, sep = "\\;")
mRNAgm$Start <- word(mRNAgm$Start, 1, sep = "\\;")
mRNAgm$End <- word(mRNAgm$End, 1, sep = "\\;")
mRNAgm$Strand <- word(mRNAgm$Strand, 1, sep = "\\;")
mRNAgm$Start <- as.numeric(mRNAgm$Start)
mRNAgm$End <- as.numeric(mRNAgm$End)

eRNAimr$Chr <- word(eRNAimr$Chr, 1, sep = "\\;")
eRNAimr$Start <- word(eRNAimr$Start, 1, sep = "\\;")
eRNAimr$End <- word(eRNAimr$End, 1, sep = "\\;")
eRNAimr$Strand <- word(eRNAimr$Strand, 1, sep = "\\;")
eRNAimr$Start <- as.numeric(eRNAimr$Start)
eRNAimr$End <- as.numeric(eRNAimr$End)

eRNAgm$Chr <- word(eRNAgm$Chr, 1, sep = "\\;")
eRNAgm$Start <- word(eRNAgm$Start, 1, sep = "\\;")
eRNAgm$End <- word(eRNAgm$End, 1, sep = "\\;")
eRNAgm$Strand <- word(eRNAgm$Strand, 1, sep = "\\;")
eRNAgm$Start <- as.numeric(eRNAgm$Start)
eRNAgm$End <- as.numeric(eRNAgm$End)

#filter by protein coding genes (for mRNA counts)
library(biomaRt)
library(dplyr)
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
gb <- getBM(attributes = c("hgnc_symbol", "gene_biotype", "entrezgene_id"), mart = ensembl)
colnames(gb)[3] <- "GeneID"
mRNAimr <- left_join(mRNAimr, gb, by = "GeneID")
mRNAimr <- subset(mRNAimr, gene_biotype == "protein_coding")
mRNAimr$gene_biotype <- NULL
mRNAimr <- distinct(mRNAimr, GeneID, .keep_all = T)

mRNAgm <- left_join(mRNAgm, gb, by = "GeneID")
mRNAgm <- subset(mRNAgm, gene_biotype == "protein_coding")
mRNAgm$gene_biotype <- NULL
mRNAgm <- distinct(mRNAgm, GeneID, .keep_all = T)

#combine regions of overlapping fragments or fragments within 500 bp of each other
library(GenomicRanges)
e_regionsIMR <- eRNAimr[, 13:17]
e_regionsIMR$Start <- e_regionsIMR$Start - 250
e_regionsIMR$End <- e_regionsIMR$End + 250
GR <- makeGRangesFromDataFrame(e_regionsIMR, keep.extra.columns = T)
e_regionsIMR <- reduce(GR, ignore.strand = F)
e_regionsIMR <- as.data.frame(e_regionsIMR)
e_regionsIMR$start <- e_regionsIMR$start + 250
e_regionsIMR$end <- e_regionsIMR$end - 250

e_regionsGM <- eRNAgm[, 13:17]
e_regionsGM$Start <- e_regionsGM$Start - 250
e_regionsGM$End <- e_regionsGM$End + 250
GR <- makeGRangesFromDataFrame(e_regionsGM, keep.extra.columns = T)
e_regionsGM <- reduce(GR, ignore.strand = F)
e_regionsGM <- as.data.frame(e_regionsGM)
e_regionsGM$start <- e_regionsGM$start + 250
e_regionsGM$end <- e_regionsGM$end - 250

#combining and renaming all eRNAs between GM and IMR
library(GenomicRanges)
all_eRNA <- rbind(e_regionsIMR, e_regionsGM)
all_eRNA$Geneid <- NULL
GR <- makeGRangesFromDataFrame(all_eRNA, keep.extra.columns = T)
all_eRNA <- reduce(GR, ignore.strand = F)
all_eRNA <- as.data.frame(all_eRNA)
all_eRNA$width <- NULL
all_eRNA$seq <- ave(all_eRNA$start, all_eRNA$seqnames, FUN = seq_along)
all_eRNA$Geneid <- paste0(all_eRNA$seqnames, "-", all_eRNA$seq, all_eRNA$strand)

#map combined regions back to original count file
GR <- makeGRangesFromDataFrame(eRNAimr, keep.extra.columns = T)
GR1 <- makeGRangesFromDataFrame(all_eRNA, keep.extra.columns = T)
pairs <- mergeByOverlaps(GR, GR1, type = "any", select = "all", ignore.strand = F)
pairs1 <- data.frame(matrix(NA, nrow = nrow(pairs[1]), ncol = 2))
pairs1[1] <- pairs[[14]]
pairs1[2] <- pairs[[18]]
colnames(pairs1) <- c("GeneID", "region")
eRNAimr <- left_join(eRNAimr, pairs1, by = "GeneID")

GR <- makeGRangesFromDataFrame(eRNAgm, keep.extra.columns = T)
pairs <- mergeByOverlaps(GR, GR1, type = "any", select = "all", ignore.strand = F)
pairs1 <- data.frame(matrix(NA, nrow = nrow(pairs[1]), ncol = 2))
pairs1[1] <- pairs[[14]]
pairs1[2] <- pairs[[18]]
colnames(pairs1) <- c("GeneID", "region")
eRNAgm <- left_join(eRNAgm, pairs1, by = "GeneID")

#provide GeneHancer ID for each annotated eRNA by overlap
library(GenomicRanges)
GH_HG19 <- readRDS("GH_HG19.rds")
GR_GH <- makeGRangesFromDataFrame(GH_HG19, keep.extra.columns = T)
GR <- makeGRangesFromDataFrame(eRNAimr, keep.extra.columns = T)
GHpairs <- mergeByOverlaps(GR, GR_GH, type = "any", select = "all", ignore.strand = T)
GH_pairsIMR <- data.frame(matrix(NA, nrow = nrow(GHpairs[1]), ncol = 2))
colnames(GH_pairsIMR) <- c("GeneID", "GeneHancerID")
GH_pairsIMR$GeneID <- GHpairs[[14]]
GH_pairsIMR$GeneHancerID <- GHpairs[[22]]

GR <- makeGRangesFromDataFrame(eRNAgm, keep.extra.columns = T)
GHpairs <- mergeByOverlaps(GR, GR_GH, type = "any", select = "all", ignore.strand = T)
GH_pairsGM <- data.frame(matrix(NA, nrow = nrow(GHpairs[1]), ncol = 2))
colnames(GH_pairsGM) <- c("GeneID", "GeneHancerID")
GH_pairsGM$GeneID <- GHpairs[[14]]
GH_pairsGM$GeneHancerID <- GHpairs[[22]]

#add GH ID to count file
eRNAimr <- left_join(eRNAimr, GH_pairsIMR, by = "GeneID")
eRNAimr$GeneHancerID <- ifelse(is.na(eRNAimr$GeneHancerID), NA , paste0(eRNAimr$GeneHancerID, eRNAimr$Strand))

eRNAgm <- left_join(eRNAgm, GH_pairsGM, by = "GeneID")
eRNAgm$GeneHancerID <- ifelse(is.na(eRNAgm$GeneHancerID), NA , paste0(eRNAgm$GeneHancerID, eRNAgm$Strand))

#rename groHMM tx to new GH ID or region ID
eRNAimr$GeneID <- ifelse(is.na(eRNAimr$GeneHancerID), eRNAimr$region, eRNAimr$GeneHancerID)
eRNAimr[, 19:21] <- NULL

eRNAgm$GeneID <- ifelse(is.na(eRNAgm$GeneHancerID), eRNAgm$region, eRNAgm$GeneHancerID)
eRNAgm[, 19:21] <- NULL

#collapse and sum counts by region
df1 <- eRNAimr[, 13:17]
df1 <- df1 %>% group_by(GeneID, Chr, Strand) %>% summarize(Start = min(Start), End = max(End))
df1$Length <- df1$End - df1$Start
df2 <- eRNAimr[, 1:13]
df2 <- df2 %>% group_by(GeneID) %>% summarize_each(function(x)sum(x))
eRNAimr <- inner_join(df1, df2, by = "GeneID")
eIMR_coord <- eRNAimr[, 1:6]
rm(df1, df2)

df1 <- eRNAgm[, 13:17]
df1 <- df1 %>% group_by(GeneID, Chr, Strand) %>% summarize(Start = min(Start), End = max(End))
df1$Length <- df1$End - df1$Start
df2 <- eRNAgm[, 1:13]
df2 <- df2 %>% group_by(GeneID) %>% summarize_each(function(x)sum(x))
eRNAgm <- inner_join(df1, df2, by = "GeneID")
eGM_coord <- eRNAgm[, 1:6]
rm(df1, df2)

#TPM calculation
eIMRtpm <- eRNAimr
eIMRtpm[,7:18] <- lapply(eIMRtpm[,7:18], function(x) ((x/(eIMRtpm$Length/1000))))
eIMRtpm[,7:18] <- lapply(eIMRtpm[,7:18], function(x) ((x/(sum(x)/1000000))))
#colSums(eIMRtpm[7:18])

mIMRtpm <- mRNAimr
mIMRtpm[,1:12] <- lapply(mIMRtpm[,1:12], function(x) ((x/(mIMRtpm$Length/1000))))
mIMRtpm[,1:12] <- lapply(mIMRtpm[,1:12], function(x) ((x/(sum(x)/1000000))))
#colSums(mIMRtpm[1:12])

eGMtpm <- eRNAgm
eGMtpm[,7:18] <- lapply(eGMtpm[,7:18], function(x) ((x/(eGMtpm$Length/1000))))
eGMtpm[,7:18] <- lapply(eGMtpm[,7:18], function(x) ((x/(sum(x)/1000000))))
#colSums(eGMtpm[7:18])

mGMtpm <- mRNAgm
mGMtpm[,1:12] <- lapply(mGMtpm[,1:12], function(x) ((x/(mGMtpm$Length/1000))))
mGMtpm[,1:12] <- lapply(mGMtpm[,1:12], function(x) ((x/(sum(x)/1000000))))
#colSums(mGMtpm[1:12])

#reorder counts columns to be in time order
eRNAimr <- eRNAimr[, c(1:7, 13, 10, 12, 15, 16, 18, 8, 9, 11, 14, 17)]
mRNAimr <- mRNAimr[, c(13:19, 1,7,4,6,9,10,12,2,3,5,8,11)]
eRNAgm <- eRNAgm[, c(1:7, 13, 10, 12, 15, 16, 18, 8, 9, 11, 14, 17)]
mRNAgm <- mRNAgm[, c(13:19, 1,7,4,6,9,10,12,2,3,5,8,11)]


#DESEQ2 NORMALIZATION OF COUNTS FOR DE ANALYSIS
#count data less timepoints > 24H
eIMRct <- eRNAimr[, c(1, 7:16)]
eIMRct <- data.frame(eIMRct, row.names = 1)
mIMRct <- mRNAimr[, c(1, 8:17)]
mIMRct <- data.frame(mIMRct, row.names = 1)

eGMct <- eRNAgm[, c(1, 7:16)]
eGMct <- data.frame(eGMct, row.names = 1)
mGMct <- mRNAgm[, c(1, 8:17)]
mGMct <- data.frame(mGMct, row.names = 1)

#sample files (required for DESeq object)
colsIMR <- colnames(eIMRct)
samps <- c("control", "treated", "treated", "treated", "treated", "treated", "treated", "treated", "treated", "treated")
coldataIMR <- data.frame(colsIMR, "condition" = samps)
coldataIMR <- data.frame(coldataIMR, row.names = 1)

colsGM <- colnames(eGMct)
#samps <- c("control", "treated", "treated", "treated", "treated", "treated", "treated", "treated", "treated", "treated")
coldataGM <- data.frame(colsGM, "condition" = samps)
coldataGM <- data.frame(coldataGM, row.names = 1)

#convert to DESeq object and apply normalization - FOR DE ANALYSIS ONLY!
library(DESeq2)
dds_eIMR <- DESeqDataSetFromMatrix(countData = eIMRct, colData = coldataIMR, design = ~ condition)
dds_eIMR <- estimateSizeFactors(dds_eIMR)
eIMRnorm <- as.data.frame(counts(dds_eIMR, normalized = T))
dds_mIMR <- DESeqDataSetFromMatrix(countData = mIMRct, colData = coldataIMR, design = ~ condition)
dds_mIMR <- estimateSizeFactors(dds_mIMR)
mIMRnorm <- as.data.frame(counts(dds_mIMR, normalized = T))

dds_eGM <- DESeqDataSetFromMatrix(countData = eGMct, colData = coldataGM, design = ~ condition)
dds_eGM <- estimateSizeFactors(dds_eGM)
eGMnorm <- as.data.frame(counts(dds_eGM, normalized = T))
dds_mGM <- DESeqDataSetFromMatrix(countData = mGMct, colData = coldataGM, design = ~ condition)
dds_mGM <- estimateSizeFactors(dds_mGM)
mGMnorm <- as.data.frame(counts(dds_mGM, normalized = T))

#remove zeros and calculate |FC| based on min/max for each gene. 
#also filter for FC > 1
#DOUBLE CHECK SUBSETS WITH GROHMM DATA
eIMRnorm[eIMRnorm == 0] <- NA
eIMRnorm$L2FC <- log2((do.call(pmax, c(eIMRnorm[, 1:10], na.rm = T)))/(do.call(pmin, c(eIMRnorm[, 1:10], na.rm = T))))
eIMRnorm <- subset(eIMRnorm,L2FC > 1)
mIMRnorm[mIMRnorm == 0] <- NA
mIMRnorm$L2FC <- log2((do.call(pmax, c(mIMRnorm[, 1:10], na.rm = T)))/(do.call(pmin, c(mIMRnorm[, 1:10], na.rm = T))))
mIMRnorm <- subset(mIMRnorm,L2FC > 1)

eGMnorm[eGMnorm == 0] <- NA
eGMnorm$L2FC <- log2((do.call(pmax, c(eGMnorm[, 1:10], na.rm = T)))/(do.call(pmin, c(eGMnorm[, 1:10], na.rm = T))))
eGMnorm <- subset(eGMnorm,L2FC > 1)
mGMnorm[mGMnorm == 0] <- NA
mGMnorm$L2FC <- log2((do.call(pmax, c(mGMnorm[, 1:10], na.rm = T)))/(do.call(pmin, c(mGMnorm[, 1:10], na.rm = T))))
mGMnorm <- subset(mGMnorm,L2FC > 1)

#determine up or down-regulated based on slope of linear regression line
slope <- function(x) {
  tempDF <- data.frame(x, time = 7:18)
  lm(x ~ time, data = tempDF, na.action = na.exclude)$coefficients[2]
}

#these all need to be double checked for the groHMM data
tdata <- as.data.frame(t(eIMRnorm))
eIMRnorm$slope <- sapply(tdata[7:18,], slope) 
eIMRnorm$DE <- ifelse(eIMRnorm$slope > 0, "UP", "DOWN")
eIMR_DEG <- eIMRnorm[, c(1, 11, 13)]
tdata <- as.data.frame(t(mIMRnorm))
mIMRnorm$slope <- sapply(tdata[7:18,], slope)
mIMRnorm$DE <- ifelse(mIMRnorm$slope > 0, "UP", "DOWN")

tdata <- as.data.frame(t(eGMnorm))
eGMnorm$slope <- sapply(tdata[7:18,], slope) 
eGMnorm$DE <- ifelse(eGMnorm$slope > 0, "UP", "DOWN")
eGM_DEG <- eGMnorm[, c(1, 11, 13)]
tdata <- as.data.frame(t(mGMnorm))
mGMnorm$slope <- sapply(tdata[7:18,], slope)
mGMnorm$DE <- ifelse(mGMnorm$slope > 0, "UP", "DOWN")
rm(tdata)

#continuity index
#Peng's script for continuity index
#this function works for smaller datasets, if you get a memory error, use function below
CI <- function(x) {
  return(diag(cor(t(x[, -ncol(x)]), t(x[, -1]), method = "spearman")))
}

#CI for very large datasets
CI2 <- function(x) {
  tx1 <- t(x[, -ncol(x)])
  tx2 <- t(x[, -1])
  cors <- c()
  for(i in 1:ncol(tx1)){
    c <- cor(tx1[, i], tx2[, i], method = "spearman")
    cors <- c(cors, c)
  }
  return(cors)
}

#only test DEGs and DEEs
library(tidyr)
eIMRnorm$Geneid <- row.names(eIMRnorm)
eIMRnorm_SE <- separate_rows(eIMRnorm, Geneid, sep = ",")
eIMRnorm_SE <- distinct(eIMRnorm_SE, Geneid, .keep_all = T)
eIMRnorm_SE[is.na(eIMRnorm_SE)] <- 0
eIMRnorm_SE$CI <- CI2(eIMRnorm_SE[, 1:10])
ci_eIMR <- subset(eIMRnorm_SE, eIMRnorm_SE$CI > 0.2 | eIMRnorm_SE$CI < -0.2)
static_eIMR <- anti_join(eIMRnorm_SE, ci_eIMR)

mIMRnorm$Geneid <- row.names(mIMRnorm)
mIMRnorm[is.na(mIMRnorm)] <- 0
mIMRnorm$CI <- CI(mIMRnorm[, 1:10])
ci_mIMR <- subset(mIMRnorm, mIMRnorm$CI > 0.2 | mIMRnorm$CI < -0.2)

ieIMR <- subset(eIMR_coord, GeneID %in% ci_eIMR$Geneid) ##coordinates for all inducible eRNAs
seIMR <- subset(eIMR_coord, GeneID %in% static_eIMR$Geneid) ##coordinates for all static eRNAs

eGMnorm$Geneid <- row.names(eGMnorm)
eGMnorm_SE <- separate_rows(eGMnorm, Geneid, sep = ",")
eGMnorm_SE <- distinct(eGMnorm_SE, Geneid, .keep_all = T)
eGMnorm_SE[is.na(eGMnorm_SE)] <- 0
eGMnorm_SE$CI <- CI2(eGMnorm_SE[, 1:10])
ci_eGM <- subset(eGMnorm_SE, eGMnorm_SE$CI > 0.2 | eGMnorm_SE$CI < -0.2)
static_eGM <- anti_join(eGMnorm_SE, ci_eGM)

mGMnorm$Geneid <- row.names(mGMnorm)
mGMnorm[is.na(mGMnorm)] <- 0
mGMnorm$CI <- CI(mGMnorm[, 1:10])
ci_mGM <- subset(mGMnorm, mGMnorm$CI > 0.2 | mGMnorm$CI < -0.2)

ieGM <- subset(eGM_coord, GeneID %in% ci_eGM$Geneid) ##coordinates for all inducible eRNAs
seGM <- subset(eGM_coord, GeneID %in% static_eGM$Geneid) ##coordinates for all static eRNAs

#add gene symbol to mRNA lists
gb$GeneID <- as.character(gb$GeneID)
ci_mIMR <- left_join(ci_mIMR, gb[,c(1,3) ], by = c("Geneid" = "GeneID"))
ci_mGM <- left_join(ci_mGM, gb[,c(1,3) ], by = c("Geneid" = "GeneID"))

#find overlapping eRNA's for DE genes (EP PAIRS) - within 200kb
#Downloaded GTF data from UCSC on Refseq genes for HG19
library(rtracklayer)
hg19gtf <- as.data.frame(rtracklayer::import("hg19.ncbiRefSeq.gtf"))

mIMR_DEG.gtf <- subset(hg19gtf, gene_id %in% ci_mIMR$hgnc_symbol)
mIMR_DEG.gtf <- subset(mIMR_DEG.gtf, type == "transcript")
mIMR_DEG.gtf <- mIMR_DEG.gtf[order(mIMR_DEG.gtf$gene_id, -mIMR_DEG.gtf$width),]
mIMR_DEG.gtf <- distinct(mIMR_DEG.gtf, gene_id, .keep_all = T)

exp_genesIMR <- mIMR_DEG.gtf
exp_genesIMR$start <- exp_genesIMR$start - 200000
exp_genesIMR$end <- exp_genesIMR$end + 200000
GR1 <- makeGRangesFromDataFrame(exp_genesIMR, keep.extra.columns = T)
GR2 <- makeGRangesFromDataFrame(ieIMR, keep.extra.columns = T)
pairs <- mergeByOverlaps(GR1, GR2, type = "any", select = "all", ignore.strand = T)
pairsIMR <- data.frame(matrix(NA, nrow = nrow(pairs[1]), ncol = 2))
pairsIMR[1] <- pairs[[6]]
pairsIMR[2] <- pairs[[12]]
colnames(pairsIMR) <- c("gene", "eRNA")
pairsIMR_combo <- pairsIMR %>% group_by(gene) %>% mutate(rn = row_number()) %>% spread(rn, eRNA)
pairsIMR_combo <- as.data.frame(pairsIMR_combo)

mGM_DEG.gtf <- subset(hg19gtf, gene_id %in% ci_mGM$hgnc_symbol)
mGM_DEG.gtf <- subset(mGM_DEG.gtf, type == "transcript")
mGM_DEG.gtf <- mGM_DEG.gtf[order(mGM_DEG.gtf$gene_id, -mGM_DEG.gtf$width),]
mGM_DEG.gtf <- distinct(mGM_DEG.gtf, gene_id, .keep_all = T)

exp_genesGM <- mGM_DEG.gtf
exp_genesGM$start <- exp_genesGM$start - 200000
exp_genesGM$end <- exp_genesGM$end + 200000
GR1 <- makeGRangesFromDataFrame(exp_genesGM, keep.extra.columns = T)
GR2 <- makeGRangesFromDataFrame(ieGM, keep.extra.columns = T)
pairs <- mergeByOverlaps(GR1, GR2, type = "any", select = "all", ignore.strand = T)
pairsGM <- data.frame(matrix(NA, nrow = nrow(pairs[1]), ncol = 2))
pairsGM[1] <- pairs[[6]]
pairsGM[2] <- pairs[[12]]
colnames(pairsGM) <- c("gene", "eRNA")
pairsGM_combo <- pairsGM %>% group_by(gene) %>% mutate(rn = row_number()) %>% spread(rn, eRNA)
pairsGM_combo <- as.data.frame(pairsGM_combo)

save.image("GM_IMR_gHMM.RData")

write.table(ci_mIMR, "imIMR_hmm.txt", sep = "\t", col.names = T, row.names = F)
write.table(ci_mGM, "imGM_hmm.txt", sep = "\t", col.names = T, row.names = F)

getwd()
list.files()

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Old packages: 'AnnotationDbi', 'AnnotationHub', 'BiocParallel', 'Biostrings',
  'caret', 'data.table', 'desc', 'fansi', 'GenomeInfoDb', 'googlesheets4',
  'knitr', 'later', 'MatrixGenerics', 'mime', 'rcmdcheck', 'RcppArmadillo',
  'readr', 'recipes', 'remotes', 'Rhdf5lib', 'RSQLite', 'S4Vectors', 'shiny',
  'tibble', 'tidymodels', 'tidyr', 'tinytex', 'xfun'

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)

Warning message:
“package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'TxDb.Hsapiens.UCSC.hg19.knownGene'”
Old packages: 'Annot

GRanges object with 0 ranges and 0 metadata columns:
   seqnames    ranges strand
      <Rle> <IRanges>  <Rle>
  -------
  seqinfo: no sequences

GRanges object with 0 ranges and 0 metadata columns:
   seqnames    ranges strand
      <Rle> <IRanges>  <Rle>
  -------
  seqinfo: no sequences

ERROR: Error in `[[<-`(`*tmp*`, name, value = "-"): 1 elements in value to replace 0 elements


In [29]:
traceback()

No traceback available 


In [35]:
getwd()

[1] "/home/jovyan/work"